<a href="https://colab.research.google.com/github/JGB-colab/Intro-Data-Analysis/blob/main/MinhaUFC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraping( Projeto Minha UFC)





<h3> Será feito um web scraping para o Projeto Minha UFC, cujo objetivo centralizar as informações por meio de um chatbot com entendimento de linguagem natural, sendo assim, cada um dos tópicos irão cobrir o que o chatbot será capaz de responder através do site.

# Observações


#Bibliotecas

In [ ]:
# Para juntar o cardápio do dia junto a api rest
!pip install flask-restful
!pip install flask-ngrok
!pip install flask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import json
import re
import time
import requests
import pickle
from collections import defaultdict
from bs4 import BeautifulSoup as bs

# Cardápio do dia

## Pici

In [ ]:
def requisicao_site(url):
    response = requests.get(url)
    content = response.content
    site = bs(content,'html.parser')
    return site
    
url = "https://www.ufc.br/restaurante/cardapio/1-restaurante-universitario-de-fortaleza/"

response = requests.get(url)
content_cardapio = response.content
site = bs(content_cardapio,'html.parser') # passando html

# Pegando os elementos do cardapio
desjejum = site.find('table',attrs={'class':'refeicao desjejum'})
almoco = site.find('table',attrs={'class':'refeicao almoco'})
jantar = site.find('table',attrs={'class':'refeicao jantar'})


In [ ]:
desjejum

<table class="refeicao desjejum">
<tbody class="listras"> <tr class="item">
<td class="bebidas">Bebidas</td>
<td class="bebidas"><span class="desc">Café</span><br/><span class="desc">Leite Integral</span><span class="desc lactose"> (Contém Lactose)</span><br/><span class="desc">Leite de Soja</span> </td>
</tr> <tr class="item">
<td class="paes">Pães</td>
<td class="paes"><span class="desc">Pão Carioca</span><span class="desc gluten"> (Contém Glúten)</span><br/><span class="desc">Pão Sovado  </span><span class="desc gluten"> (Contém Glúten)</span> </td>
</tr> <tr class="item">
<td class="frutas">Frutas</td>
<td class="frutas"><span class="desc">Maçã</span><br/><span class="desc">Abacaxi</span> </td>
</tr> <tr class="item">
<td class="especial">Especial</td>
<td class="especial"><span class="desc">Carne moída ao molho</span> </td>
</tr> </tbody>
</table>

In [ ]:
def junta_palavras(vetor_string):
    texto = ''
    for elemento in vetor_string:
      texto += elemento +','
    return texto[0:-1]

In [ ]:
# Retornando cardapio em formato dicionário
def cardapio_do_dia():
    dia = site.find('table').findAll('th')[1].text.split() # Dia da semana
    cardapio_json = {"restaurante_menu":[{"Dia": "{0}{1}".format(dia[0],dia[1])},{"Campi":"Pici"}]}
    desjejum_list= junta_palavras([i.text for i in desjejum('span')])
    almoco_list = junta_palavras([i.text for i in almoco('span')]) 
    jantar_list = junta_palavras([i.text for i in jantar('span')]) 
    cardapio_json["restaurante_menu"].append({"Desjejum": json.dumps((desjejum_list),ensure_ascii=False)})
    cardapio_json["restaurante_menu"].append({"Almoco":  json.dumps(almoco_list,ensure_ascii=False)})
    cardapio_json["restaurante_menu"].append({"Jantar": json.dumps((jantar_list),ensure_ascii=False)})
    return json.dumps(cardapio_json,ensure_ascii = False)
    

In [ ]:
# Checando erro de retorno do cardápio,para saber se existe ou não cardápio no dia
def checagem_de_retorno():
  try:
    menu = json.loads(cardapio_do_dia())
    aux = 0
  except:
      dia = site.find('table').findAll('th')[1].text.split()
      menu = dia[0]+dia[1]+': '+ site.find('td',attrs = {'colspan':'3'}).text  # Mostra mensagem de não haver cardápio
      aux = 1
  return [menu,aux]

In [ ]:
checagem_de_retorno()

[{'restaurante_menu': [{'Dia': 'Sexta(21/10)'},
   {'Campi': 'Pici'},
   {'Desjejum': '"Café,Leite Integral, (Contém Lactose),Leite de Soja,Pão Carioca, (Contém Glúten),Pão Sovado  , (Contém Glúten),Maçã,Abacaxi,Carne moída ao molho"'},
   {'Almoco': '"Dobradinha,Filé de frango acebolado,Yakissoba Vegetariano, (Contém Glúten),Batata sautè com orégano,Cuscuz,Arroz Branco,Arroz Integral,Feijão de Corda,Melão espanhol,Doce"'},
   {'Jantar': '"Calabresa acebolada,Escondidinho de frango, (Contém Lactose),Tortinha de Legumes, (Contém Glúten),Alface, Repolho, Cenoura, Salsinha,Farofa Crispy,Arroz Branco,Arroz Integral,Feijão Carioca,Banana,Doce"'}]},
 0]

## Quixadá

In [ ]:
def cardapio_quixada():
    try: 
      ru_quixada = requisicao_site('https://www.quixada.ufc.br/restaurante-universitario/')
      cardapio = ru_quixada.findAll('img',attrs={'loading':'lazy'})
      almoco = cardapio[0]['srcset'].split(',')
      jantar = cardapio[1]['srcset'].split(',')
      link_almoco = almoco[0].split(' ')
      link_jantar = jantar[0].split(' ')
      cardapio_quixada = [{ "cardapio":[{'almoco': link_almoco[0]},{'jantar':link_jantar[0]}]},'image']
    except ConnectionError:
       cardapio_quixada = [{ "cardapio":[{'almoco': 'Houve um erro de conexão ao tentar requisitar o almoço'},{'jantar':'Houve um erro de conexão ao tentar requisitar o almoço'}]},'text']
    return json.dumps(cardapio_quixada,ensure_ascii = False)
json_quixada = cardapio_quixada()

In [ ]:
cardapio_quixada = json.loads(json_quixada)

In [ ]:
cardapio_quixada

[{'cardapio': [{'almoco': 'https://www.quixada.ufc.br/wp-content/uploads/2022/10/Cardapio-almoco-e-jantar-17-a-21-de-outubro_page-0001-1024x725.jpg'},
   {'jantar': 'https://www.quixada.ufc.br/wp-content/uploads/2017/09/37172224072_5b2023c322_h-1-1024x680.jpg'}]},
 'image']

In [ ]:
# ----------------Esboço de como a função em javascript deve ser-----------------------#

def mostrar_menu():
  ##Caso nao tenha cardápio
  if desjejum == None:
        dia = site.find('table').findAll('th')[1].text.split()
        print(dia[0]+dia[1]+'\n'+site.find('td',attrs = {'colspan':'3'}).text)   # Mostra mensagem de não haver cardápio
  else:
    # transformando tipo DICIONÁRIO em formato json
    cardapio = cardapio_do_dia()
    json_menu = cardapio
      # Retornando cardapio
    print(json.loads(json_menu)['restaurante_menu'][0]['Dia']
    + '\n')
    print('Campi',':',json.loads(json_menu)['restaurante_menu'][1]['Campi'])
    print('\n')
    print('Desjejum:\n')
    print(json.loads(json_menu)['restaurante_menu'][2]['Desjejum'])
    print('\n')
    print('Almoço:\n')
    print(json.loads(json_menu)['restaurante_menu'][3]['Almoco'])
    print('\n')
    print('Jantar:\n')
    print('\n')
    print(json.loads(json_menu)['restaurante_menu'][4]['Jantar'])

# Perguntas Frequentes

###Teste unitário para uma página do faq

In [ ]:
#content_faq = requisicao_site('https://prograd.ufc.br/pt/perguntas-frequentes/?limit=todas')

In [ ]:
# Armezando perguntas em um dicionário 

#2) Requisitando da página do faq, as perguntas frequentes
#def pega_conteudo(tag1,tag2):
#lista_de_perguntas = content_faq.find('div',{'class':'subpages'}).find('ul',{'class':'menu'}) # objeto bs4

# 3) Armezenando nomes em dicionário
#armazena_nomes = {}
# j = 0 
#  for pergunta in lista_de_perguntas:  
#    armazena_nomes[str(pergunta.string)] = pergunta.find(tag, {'href':re.compile('^https')}) # Esta linha encontra qualquer objeto começando com https dentro do objeto bs4

## Executando coleta de conteudo de perguntas frequentes

In [ ]:
def coleta_dados_faq(armazena_nomes,topico):
  lista_faq = {}
  for topico in armazena_nomes:
    link = armazena_nomes[str(topico)]['href']
    html = requisicao_site(link)
    conteudo = html.find('article',{'class':'article'})
    lista_faq[str(topico)] = conteudo
    print(lista_faq)
    time.sleep(60)
  return lista_faq

In [ ]:
#armazena_nomes

In [ ]:
#admissao= requisicao_site(armazena_nomes['Admissão de Graduados']['href']) # Testando função de requisicao para um elemento da lista


## PRAE

In [ ]:
site_prae = requisicao_site('https://prae.ufc.br/pt/')

In [ ]:
padrao  = re.compile(r'\d{4}') # $ 4 dígitos quaisquer $

In [ ]:
conteudo_info = site_prae.findAll('li')
links_prae = []
lista_nomes = []
for element in conteudo_info:
    try:
      links_prae.append({element.get_text(): element.contents[0]['href']})
      lista_nomes.append(element.get_text())
    except TypeError:
      pass

In [ ]:
'''conteudo_faq_prae = []
for elemento in links_prae[5:]:
    link = list(elemento.values())[0]
    nome = list(elemento.keys())[0]
    requisicao = requisicao_site(link)
    conteudo_faq_prae.append({nome:requisicao})
    time.sleep(30)'''

'conteudo_faq_prae = []\nfor elemento in links_prae[5:]:\n    link = list(elemento.values())[0]\n    nome = list(elemento.keys())[0]\n    requisicao = requisicao_site(link)\n    conteudo_faq_prae.append({nome:requisicao})\n    time.sleep(30)'

In [ ]:
'''def pega_texto():
  respostas_faq_prae = defaultdict(list)
  for ind in range(2,len(conteudo_faq_prae)):
    html = conteudo_faq_prae[ind][lista_nomes[ind+5]]
    resposta = html.findAll('p')
    for parte in resposta:
      texto = ''
      texto += parte.text +'\n'
      respostas_faq_prae[lista_nomes[ind+5]].append(texto)
  return respostas_faq_prae'''

"def pega_texto():\n  respostas_faq_prae = defaultdict(list)\n  for ind in range(2,len(conteudo_faq_prae)):\n    html = conteudo_faq_prae[ind][lista_nomes[ind+5]]\n    resposta = html.findAll('p')\n    for parte in resposta:\n      texto = ''\n      texto += parte.text +'\n'\n      respostas_faq_prae[lista_nomes[ind+5]].append(texto)\n  return respostas_faq_prae"

In [ ]:
#respostas = pega_texto()

In [ ]:
#respostas

In [ ]:
conteudo_prae= defaultdict(list,
            {'Ajuda de Custo': ['O Programa Ajuda de Custo tem por objetivos:\n',
              '1. Conceder ajuda de custo aos estudantes dos Cursos de Graduação da Universidade Federal do Ceará (UFC) que desejam participar em eventos de natureza acadêmica,\xa0 artística ou de movimentos estudantis;\n',
              '2. Conceder ajuda de custo aos estudantes que representam o Diretório Central dos Estudantes (DCE), os Centros Acadêmicos (CA’s) e as Associações Atléticas na participação em eventos do movimento estudantil, com representação de delegações e na promoção de eventos acadêmicos, políticos, artísticos e esportivos.\n',
              '3. Conceder ajuda de custo aos estudantes que representam as seleções esportivas da Universidade Federal do Ceará em competições, locais, regionais, nacionais e internacionais.\n',
              'DIVISÃO DE APOIO ADMINISTRATIVO\nEndereço: Rua Paulino Nogueira, 315 – Bloco III – 1° Andar – Benfica – Fortaleza\xa0– CE\nFone: (85) 3366 7440 / 3366 7443\nFax: (85) 3366 7442\nE-mail: prae.secretaria@ufc.br\n',
              'Rua Paulino Nogueira, 315 - Bloco III - 1° Andar - Benfica - CEP 60020-270 - Fortaleza - CE\n',
              'tel: +55 (85) 3366 7440\n'],
             'Auxílio Creche': ['O Auxílio-Creche foi regulamentado por meio do anexo ad referendum XXI – da resolução nº 08/2013 do CEPE. Para acessar a este benefício, a/o discente deve estar matriculada(o) em um dos cursos de graduação da UFC, na carga horária mínima exigida no edital correspondente; apresentar situação de vulnerabilidade socioeconômica com base nos parâmetros estabelecidos no PNAES, e ter filho(a) com idade entre 6 meses e 4 anos incompletos, que esteja sob sua guarda e que coabite com a(o) mesma(o).\n',
              'No que diz respeito à renovação do benefício, esta se dá conforme as indicações dispostas nos editais. É válido ainda ressaltar, que o auxílio-creche é cumulativo com quaisquer outras bolsas, estágios ou trabalhos formais remunerados dos discentes.\n',
              'DIVISÃO DE BENEFÍCIOS E MORADIA\nEndereço: Rua Paulino Nogueira, 315 – Bloco III – 1° Andar – Benfica – Fortaleza\xa0– CE\nFone: (85) 3366 7448\nE-mail: dibem.case@ufc.br\n',
              'Rua Paulino Nogueira, 315 - Bloco III - 1° Andar - Benfica - CEP 60020-270 - Fortaleza - CE\n',
              'tel: +55 (85) 3366 7440\n'],
             'Auxílio Emergencial': ['O Auxílio Emergencial surgiu com o intuito de assistir estudantes em situação de vulnerabilidade socioeconômica, que ingressam na universidade em listas de espera do SISU, e que portanto, adentram após os processos seletivos da PRAE. Também foi originalmente pensado para atender estudantes que chegavam no segundo semestre letivo do ano, em chamada regular ou de espera. Mas com o passar do tempo passou a atender igualmente outras situações, tais como:\n',
              'Para a solicitação do benefício o/a discente deve atender pelo menos um dos requisitos estabelecidos, possuir renda familiar per capita compatível com a definida pelo PNAES e estar atento aos demais requisitos dispostos nos editais. Ressalte-se que, para solicitá-lo, os discentes não podem ser beneficiados por outras bolsas, estágios ou outras formas de rendas formais, a exceção dos residentes e beneficiários do auxílio-moradia.\n',
              'DIVISÃO DE BENEFÍCIOS E MORADIA\nEndereço: Rua Paulino Nogueira, 315 – Bloco III – 1° Andar – Benfica – Fortaleza\xa0– CE\nFone: (85) 3366 7448\nE-mail: dibem.case@ufc.br\n',
              'Rua Paulino Nogueira, 315 - Bloco III - 1° Andar - Benfica - CEP 60020-270 - Fortaleza - CE\n',
              'tel: +55 (85) 3366 7440\n'],
             'Auxílio Moradia': ['O Programa Auxílio Moradia tem por objetivo viabilizar a permanência de estudantes matriculados nos Cursos de Graduação dos Campi da Universidade Federal do Ceará (UFC) em Crateús, Quixadá, Russas e Sobral em comprovada situação de vulnerabilidade econômica, assegurando-lhes auxílio institucional para complementação de despesas com moradia e alimentação durante todo o período do curso ou enquanto persistir a mesma situação.\n',
              'Importante! A vinculação dos estudantes ao Programa Auxílio Moradia não os impede de receber, por mérito, qualquer uma das bolsas dos diversos programas da UFC, de agências de fomento ou de empresas.\n',
              'COORDENADORIA DE ASSISTÊNCIA ESTUDANTIL\nDIVISÃO DE BENEFÍCIOS E MORADIA\nFone: (85) 3366 7448\n',
              'E-mail: dibem.case@ufc.br\n',
              'Rua Paulino Nogueira, 315 - Bloco III - 1° Andar - Benfica - CEP 60020-270 - Fortaleza - CE\n',
              'tel: +55 (85) 3366 7440\n'],
             'Acompanhamento ao Estudante': ['DIVISÃO DE ATENÇÃO AO ESTUDANTE – DAE\nEndereço: Rua Paulino Nogueira, 315 – Bloco III – 1° andar – Benfica – Fortaleza – CE\nFone: (85) 3366 7447 – E-mail: dae.case@ufc.br\n',
              'A equipe da Divisão de Atenção ao Estudante (DAE), ligada à Coordenadoria de Assistência Estudantil (CASE), realiza o acompanhamento psicológico e psicossocial de estudantes com matrícula ativa em cursos de graduação da UFC Fortaleza. De 1987 até 2016, o trabalho desenvolvido hoje pela DAE era realizado pelo Programa de Apoio Psicopedagógico ao Estudante Universitário (PAPEU).\n',
              'A equipe da DAE é composta por psicólogos(as) e assistente social. São ofertados gratuitamente os seguintes serviços: Acolhimento; Atendimento Psicológico/Psicanalítico Individual; e Atividades de Grupo.\n',
              'Rua Paulino Nogueira, 315 - Bloco III - 1° Andar - Benfica - CEP 60020-270 - Fortaleza - CE\n',
              'tel: +55 (85) 3366 7440\n'],
             'Bolsa de Incentivo ao Desporto': ['O Programa Bolsa de Incentivo ao Desporto tem por objetivo incentivar os estudantes a incrementarem seu desempenho desportivo e acadêmico, mediante atuação em atividades relativas à gestão desportiva e rendimento desportivo.\n',
              'COORDENADORIA DE ATIVIDADES DESPORTIVAS\nEndereço: Rua Paulino Nogueira, 2762 – Benfica\xa0– Fortaleza\xa0– CE\nFone: (85) 3366 7871\nE-mail:desportoufc@ufc.br\n',
              'Rua Paulino Nogueira, 315 - Bloco III - 1° Andar - Benfica - CEP 60020-270 - Fortaleza - CE\n',
              'tel: +55 (85) 3366 7440\n'],
             'Bolsa de Iniciação Acadêmica': ['O Programa Bolsa de Iniciação Acadêmica tem por objetivo propiciar aos estudantes de Cursos de Graduação Presenciais da Universidade Federal do Ceará (UFC) – em situação de vulnerabilidade sócio-econômica comprovada – especialmente os de semestres iniciais, condições financeiras para sua permanência e desempenho acadêmico satisfatório, mediante atuação, em caráter de iniciação acadêmica, nas diversas unidades da Instituição.\n',
              'DIVISÃO DE BENEFÍCIOS E MORADIA\nEndereço: Rua Paulino Nogueira, 315 – Bloco III – 1º Andar – Benfica\xa0– Fortaleza\xa0– CE\nFone: (85) 3366 7448\nFax: (85) 3366 7442\nE-mail: dibem.case@ufc.br\n',
              'Rua Paulino Nogueira, 315 - Bloco III - 1° Andar - Benfica - CEP 60020-270 - Fortaleza - CE\n',
              'tel: +55 (85) 3366 7440\n'],
             'Residência Universitária': ['\n',
              'A Moradia Estudantil no âmbito da residência universitária visa contribuir para permanência do estudante em vulnerabilidade socioeconômica nos cursos presenciais de graduação, considerando que este estudante não possui núcleo familiar residindo na sede onde realiza a graduação.\n',
              'A assistência estudantil assim busca propiciar condições dignas de moradia e alimentação de qualidade para este público a fim de concluir o curso de forma satisfatória.\n',
              'Atualmente há em funcionamento 10 residências universitárias: 09 no campus Benfica, 01 no campus Pici.\n',
              'DIVISÃO DE BENEFÍCIOS E MORADIA\n',
              'Endereço: Rua Paulino Nogueira, 315 – Bloco III – 1º Andar – Benfica\xa0– Fortaleza\xa0– CE\nFone: (85) 3366 7448\nFax: (85) 3366 7442\nE-mail:dibem.case@ufc.br\n',
              'Rua Paulino Nogueira, 315 - Bloco III - 1° Andar - Benfica - CEP 60020-270 - Fortaleza - CE\n',
              'tel: +55 (85) 3366 7440\n'],
             'Restaurante Universitário': ['O Programa de Restaurante Universitário ter por objetivo oferecer refeição balanceada e de qualidade a estudantes, docentes, servidores técnico-administrativos da UFC e pessoal a serviço da UFC de empresas terceirizadas contratadas, além de constituir um espaço de convivência e integração da comunidade universitária.\n',
              'COORDENADORIA DE RESTAURANTE UNIVERSITÁRIO\nFone/Fax: (85) 3366 7441\nE-mail:ru@ufc.br\n',
              'HORÁRIOS DE FUNCIONAMENTO:\nCafé da manhã (somente para Residentes): das 6h15min às 8h30min\nAlmoço: das 11h às 14h\nJantar: das 17h às 19h30min\nRecarga dos cartões: das 08h às 19h\n',
              'VALOR DAS REFEIÇÕES:\nEstudante – R$ 1,10\nServidor técnico–administrativo – R$ 13,50\xa0 – Atualizado em Abril/2022\nDocente – R$ 13,50 – Atualizado em Abril/2022\n\n',
              'PÚBLICO ALVO: Estudantes, servidores técnico–administrativos, docentes e pessoal a serviço da UFC de empresas terceirizadas contratadas.\n',
              'Além das categorias acima descritas, também podem solicitar o acesso aos Refeitórios participantes de eventos realizados pela UFC (congressos, encontro de estudantes, cursos, etc.), desde que estejam vinculados a outras Instituições Federais de Ensino Superior, pesquisadores que estejam desenvolvendo algum trabalho em parceria com a Universidade, bolsistas de projetos, dentre outros, desde que formalizem a solicitação através do envio de um ofício para a Pró-Reitoria de Assuntos Estudantis.\n',
              'Formulário de\xa0Solicitação de Serviços do RU – Eventos de Serviços do RU (DOC 113 KB) Outros formatos: ODT (57 KB) PDF (46 KB)\n',
              'Controle Gerencial de Participantes de Eventos (DOC 145 KB) Outros formatos: ODT (58 KB) PDF (61 KB)\n',
              'Controle Gerencial de Visitantes\xa0(DOC 144 KB) Outros formatos: ODT (58 KB) PDF (61 KB)\n',
              'Rua Paulino Nogueira, 315 - Bloco III - 1° Andar - Benfica - CEP 60020-270 - Fortaleza - CE\n',
              'tel: +55 (85) 3366 7440\n'],
             'Projetos e Ações': ['A\xa0 equipe da Pró-Reitoria de Assuntos Estudantis, se reinventou e, mesmo no período de pandemia desenvolveu inúmeras atividades, ações, projetos, mudanças estruturais e implementação de novos benefícios objetivando a melhoria de seu produto e alcançar novos objetivos.\n',
              '\xa0\n',
              'Confere aqui as nossas ações inovadoras\n',
              'Ações Inovadoras da PRAE ( PDF 6 MB)\n',
              'Rua Paulino Nogueira, 315 - Bloco III - 1° Andar - Benfica - CEP 60020-270 - Fortaleza - CE\n',
              'tel: +55 (85) 3366 7440\n'],
             'Editais e Resultados': ['\xa0\n',
              'Rua Paulino Nogueira, 315 - Bloco III - 1° Andar - Benfica - CEP 60020-270 - Fortaleza - CE\n',
              'tel: +55 (85) 3366 7440\n'],
             'Endereços e Telefones': ['Pró-Reitora: Profa. Dra. Geovana Maria Cartaxo de Arruda Freire\nEndereço: Rua Paulino Nogueira, 315 – Benfica – CEP 60020-270 – Fortaleza – CE\nFone: (85) 3366 7440\nFax: (85) 3366 7442\nE-mail: prae.secretaria@ufc.br\n',
              'DIVISÃO DE APOIO ADMINISTRATIVO\nFone: (85) 3366 7440\nFax: (85) 3366 7442\nE-mail: prae.secretaria@ufc.br\n',
              'ASSESSORIA ADMINISTRATIVA\nFax: (85) 3366 7442\nE-mail: aap.prae@ufc.br\n',
              'Coordenadora: Márcia Regina Mariano de Sousa Arão\nFone: (85) 3366 7445 / 3366 7444\nFax: (85) 3366 7442\nE-mail: case@ufc.br\n',
              'DIVISÃO DE ATENÇÃO AO ESTUDANTE\nFone: (85) 3366 7447\nFax: (85) 3366 7442\nE-mail: dae.case@ufc.br\n',
              'DIVISÃO DE BENEFÍCIOS E MORADIA\nFone: (85) 3366 7448\nFax: (85) 3366 7442\nE-mail: dibem.case@ufc.br\n',
              'Coordenador: Wildner Lins de Souza\nEndereço: Avenida da Universidade, 2762 – Campus do Benfica II -Quadra do CEU ao lado do Bloco de Psicologia – Fortaleza – CE\nFone: (85) 3366 7871\nE-mail: desportoufc@ufc.br\n',
              'DIVISÃO DE DESPORTO DE PARTICIPAÇÃO\nFone: (85) 3366 7871\n',
              'DIVISÃO DE DESPORTO DE RENDIMENTO\nDiretor: José Clovandi Costa Filho\nFone: (85) 3366 7871\n',
              'Coordenador: Francisco José Albuquerque Cruz\nFone/Fax: (85) 3366 7441\nE-mail: praeufc@gmail.com\n',
              'DIVISÃO DE ALIMENTAÇÃO E NUTRIÇÃO\nDiretora: Natália Lopes Vasconcelos\nFone: (85) 3366 9531\n',
              'Seção do Refeitório do Benfica\nChefe: Conceição de Maria Coelho de Carvalho\nFone/Fax: (85) 3366 7776\n',
              'Seção do Refeitório do Pici\nChefe: Cleonice de Castro Silva\nFone/Fax: (85) 3366 9530\n',
              'Seção do Refeitório do Porangabuçu\nFone/Fax: (85) 3366 7447\n',
              'DIVISÃO DE SERVIÇOS OPERACIONAIS\nDiretor: José Pereira Duarte\nFone/Fax: (85) 3366 9530\n',
              'Rua Paulino Nogueira, 315 - Bloco III - 1° Andar - Benfica - CEP 60020-270 - Fortaleza - CE\n',
              'tel: +55 (85) 3366 7440\n'],
             'Notícias': ['Rua Paulino Nogueira, 315 - Bloco III - 1° Andar - Benfica - CEP 60020-270 - Fortaleza - CE\n',
              'tel: +55 (85) 3366 7440\n'],
             'OBSERVAÊ – Observatório da Vida Estudantil': ['\n',
              'A Pró-Reitoria de Assuntos Estudantis, no dia 26 de novembro de 2020, realizou solenidade de lançamento do Observatório da Vida Estudantil (OBSERVAÊ).\n',
              'O referido observatório objetiva sistematizar, coletar e divulgar informações pertinentes à vida do estudante da UFC, com foco em fortalecer e orientar a política de assistência estudantil, e oferecer dados e subsídios para o aperfeiçoamento dos processos de planejamento, gestão, execução e avaliação desta importante política pública de inclusão social. O Projeto contou com apoio e colaboração das diversas divisões que formam a PRAE e consta como objetivo prioritário do nosso Plano de Desenvolvimento Institucional (PDI).\n',
              'Segue portaria que instituiu o Observatório da Vida Estudantil (OBSERVAÊ)\n',
              '⇒ Portaria nº 178, de 25 de novembro de 2020, do Gabinete do Reitor\n',
              'Confira também a notícia e o webnário do seu lançamento na íntegra:\n',
              '⇒ Página da UFC (temporariamente indisponível em razão do período defeso eleitoral)\n',
              '⇒ Galeria de fotos (temporariamente indisponível em razão do período defeso eleitoral)\n',
              '⇒ Webnário “Assistência Estudantil: avaliação e fortalecimento da política nas instituições de nível superior”:\n',
              '\n',
              'Rua Paulino Nogueira, 315 - Bloco III - 1° Andar - Benfica - CEP 60020-270 - Fortaleza - CE\n',
              'tel: +55 (85) 3366 7440\n'],
             'MapSaúde – Promoção da Qualidade de Vida Estudantil': ['Esta iniciativa objetiva coletar, organizar e divulgar dados sobre projetos e ações internos e externos à UFC que contribuam para a Qualidade de Vida dos estudantes.\n',
              'As exigências decorrentes da rotina acadêmica (conteúdos curriculares, avaliações, apresentação de seminários, habilidades sociais, etc.), aliadas a fatores como vulnerabilidade socioeconômica e diversas fragilidades psíquicas, podem levar a condições de tensão, angústia e até quadros de adoecimento.\n',
              'Tais questões, identificadas pela equipe técnica da Pró-Reitoria de Assuntos Estudantis (PRAE) nos seus afazeres diários, somadas à impossibilidade de suprir a demanda por atendimento psicológico, levaram à formulação deste Projeto denominado Mapsaúde.\n',
              'Os interessados em contribuir com o Mapsaúde, divulgando projetos e ações relativos à qualidade de vida estudantil, podem entrar em contato conosco via e-mail: mapsaude@ufc.br ou através do instagram @mapsaudeufc. Agradecemos a parceria.\n',
              'Listados abaixo estão os projetos mapeados até o momento que se adequaram para funcionar durante o período da pandemia:\n',
              'Rua Paulino Nogueira, 315 - Bloco III - 1° Andar - Benfica - CEP 60020-270 - Fortaleza - CE\n',
              'tel: +55 (85) 3366 7440\n'],
             'Agenda da Pró-reitora': ['\n',
              'Rua Paulino Nogueira, 315 - Bloco III - 1° Andar - Benfica - CEP 60020-270 - Fortaleza - CE\n',
              'tel: +55 (85) 3366 7440\n'],
             'Intimações': ['Ficam intimados os estudantes abaixo listados, \xa0conforme determina o Art. 26, § 4º, da Lei no 9.784/99, a comparecer à Pró- Reitoria de Assuntos Estudantis, da Universidade Federal do Ceará (UFC), situada à Rua Paulino Nogueira, n 315- Fortaleza – CE, das 08:00h às 17:00h, a fim de cumprir o que determina cada Edital, os quais podem ser acessados clicando no respectivo links\n',
              '\xa0\n',
              'MARIA DEANE SANTOS\n',
              'JOAO VICTOR ARAGÃO ALVES 18/08/2022\n',
              '\xa0\n',
              'JOAO VICTOR ARAGÃO ALVES 05/07/2022\n',
              '\xa0\n',
              'Milena Bezerra Gomes 25/04/2022\n',
              'Lucivania Santos Lima 25/04/2022\n',
              'Eric Alves Mendes 25/04/2022\n',
              'Carine de Mesquita Barbosa 25/04/2022\n',
              '\xa0\n',
              '\xa0\n',
              'Ian costa fernandes 14/02/2022\n',
              'Jonathan de sousa paiva 14/02/2022\n',
              'Renato gomes da costa 14/02/2022\n',
              'Alysson de oliveira andrade 14/02/2022\n',
              'Bruno pereira de almeida 14/02/2022\n',
              '\xa0\n',
              '\xa0\n',
              'Francisco Herivelton Peixoto Cardoso 07/02/2022\n',
              '\xa0\n',
              'Vanessa Venancio de Sousa 04/02/2022\n',
              '\xa0\n',
              'Humberto Goncalves Meireles 02/02/2022\n',
              '\xa0\n',
              'Yoanes Maciel de Sousa 21/12/2021\n',
              'Thiago Silva Duarte 21/12/2021\n',
              'Ramon Mota de Sousa 21/12/2021\n',
              'Raireno Pinheiro Maciel 21/12/2021\n',
              'Maria Eduarda Sousa Lopes 21/12/2021\n',
              'Debora Cristina Olivindo da Silva 21/12/2021\n',
              '\xa0\n',
              '\xa0\n',
              'ANTONIO JOSUELDO APRIGIO DE SOUZA 05/11/2021\n',
              'KATIANE FERREIRA DA SILVA 05/11/2021\n',
              'PAULO VICTOR SOMBRA BOMFIM\xa0 05/11/2021\n',
              'FRANCISCA ELAINE DE SOUZA FRANÇA 28/10/2021\n',
              'Rua Paulino Nogueira, 315 - Bloco III - 1° Andar - Benfica - CEP 60020-270 - Fortaleza - CE\n',
              'tel: +55 (85) 3366 7440\n'],
             0: [],
             1: []})

In [ ]:
topics = list(conteudo_prae.keys())

In [ ]:
answer  =list(conteudo_prae.values())

In [ ]:
# Crindo dataframe para coleta de dados
planilha_prae = pd.DataFrame(data = {'resposta':answer},index = topics)
# Dropando linhas indesejadas
def dropar_linhas(df,*args):
    lista_drop = list(args)
    nova_planilha = df.drop(axis = 0, index = lista_drop)
    return nova_planilha
planilha_prae = dropar_linhas(planilha_prae, 'Editais e Resultados','Endereços e Telefones','Notícias','Agenda da Pró-reitora','Intimações',0,1)
planilha_prae

,resposta
Ajuda de Custo,[O Programa Ajuda de Custo tem por objetivos:\...
Auxílio Creche,[O Auxílio-Creche foi regulamentado por meio d...
Auxílio Emergencial,[O Auxílio Emergencial surgiu com o intuito de...
Auxílio Moradia,[O Programa Auxílio Moradia tem por objetivo v...
Acompanhamento ao Estudante,[DIVISÃO DE ATENÇÃO AO ESTUDANTE – DAE\nEndere...
Bolsa de Incentivo ao Desporto,[O Programa Bolsa de Incentivo ao Desporto tem...
Bolsa de Iniciação Acadêmica,[O Programa Bolsa de Iniciação Acadêmica tem p...
Residência Universitária,"[\n, A Moradia Estudantil no âmbito da residên..."
Restaurante Universitário,[O Programa de Restaurante Universitário ter p...
Projetos e Ações,[A equipe da Pró-Reitoria de Assuntos Estudan...


In [ ]:
# definindo formato de texto legível
def formata_texto(df,linhas,colunas):
    for ind in linhas:
      frase = ''
      for texto in planilha_prae.loc[ind,colunas]:
        frase = frase +  texto
      planilha_prae.loc[ind,colunas] = frase
      frase = ''
    return df
planilha_prae = formata_texto(planilha_prae,planilha_prae.index,'resposta')

In [ ]:
planilha_prae

,resposta
Ajuda de Custo,O Programa Ajuda de Custo tem por objetivos:\n...
Auxílio Creche,O Auxílio-Creche foi regulamentado por meio do...
Auxílio Emergencial,O Auxílio Emergencial surgiu com o intuito de ...
Auxílio Moradia,O Programa Auxílio Moradia tem por objetivo vi...
Acompanhamento ao Estudante,DIVISÃO DE ATENÇÃO AO ESTUDANTE – DAE\nEndereç...
Bolsa de Incentivo ao Desporto,O Programa Bolsa de Incentivo ao Desporto tem ...
Bolsa de Iniciação Acadêmica,O Programa Bolsa de Iniciação Acadêmica tem po...
Residência Universitária,\nA Moradia Estudantil no âmbito da residência...
Restaurante Universitário,O Programa de Restaurante Universitário ter po...
Projetos e Ações,A equipe da Pró-Reitoria de Assuntos Estudant...


In [ ]:
#Salvando planilha de resposta de tópicos
planilha_prae.to_excel('faq_prae.xlsx')

### Aqui houve o salvamento do arquivo então e o carregamento de outra planilha, para que se possa juntar as respostas nessa planilha carregada

In [ ]:
'''qnas_excel = pd.read_excel('/content/QnA.xlsx')  # Planilha carregada
qnas_excel.to_csv ("Test.csv",  
                  index = None, 
                  header=True)
    
qnas = pd.DataFrame(pd.read_csv("Test.csv")) 
qnas'''

'qnas_excel = pd.read_excel(\'/content/QnA.xlsx\')  # Planilha carregada\nqnas_excel.to_csv ("Test.csv",  \n                  index = None, \n                  header=True)\n    \nqnas = pd.DataFrame(pd.read_csv("Test.csv")) \nqnas'

In [ ]:
'''qnas_prae = pd.read_excel('/content/faq_prae.xlsx')''' # Planilha salva

"qnas_prae = pd.read_excel('/content/faq_prae.xlsx')"

In [ ]:
'''filtro_prae = qnas[qnas['Setor']=='prae']'''

"filtro_prae = qnas[qnas['Setor']=='prae']"

In [ ]:
'''qnas_prae.rename(columns = {'Unnamed: 0':'Assuntos'}, inplace = True)'''

"qnas_prae.rename(columns = {'Unnamed: 0':'Assuntos'}, inplace = True)"

In [ ]:
'''qnas_prae'''

'qnas_prae'

In [ ]:
'''
# Juntando os dfs
planilha_formatada = pd.DataFrame(columns= filtro_prae.columns, index = [i for i in range(len(qnas_prae))])
planilha_formatada['Resposta'] = qnas_prae['resposta']
planilha_formatada['Setor'] = pd.Series(['prae']*(len(planilha_formatada)))
planilha_formatada['Assunto (contexto)'] = qnas_prae['Assuntos']'''

"\n# Juntando os dfs\nplanilha_formatada = pd.DataFrame(columns= filtro_prae.columns, index = [i for i in range(len(qnas_prae))])\nplanilha_formatada['Resposta'] = qnas_prae['resposta']\nplanilha_formatada['Setor'] = pd.Series(['prae']*(len(planilha_formatada)))\nplanilha_formatada['Assunto (contexto)'] = qnas_prae['Assuntos']"

In [ ]:
'''planilha_formatada'''

'planilha_formatada'

In [ ]:
#resultado = pd.concat([qnas,planilha_formatada])
#resultado

In [ ]:
#resultado.to_excel('Qnas_atualizado.xlsx')

# Contatos

## Prograd(Com requisição)



<h4>Esta função possui os seguintes elementos em sua estrutura:</h4>

*  {"sector"}---> setor
*	 {"email"}---> endereço de e-mail
*	 {"phone"}---> telefone de contato
*	 {"site"}----> link do site 
*	 {"faq"} ----> link das perguntas frequentes
*	 {"contacts"} ---> link geral de contatos e endereços
*	 {"whatsapp"} ---> contato whatsapp



In [ ]:
def requisicao_site_json(url):
    response = requests.get(url)
    content = response.content
    return json.loads(content)

In [ ]:
# Usando a função de requisição do faq temos
site_prograd = requisicao_site('https://prograd.ufc.br/pt/contatos/')

In [ ]:
#Puxando links dos setores
setores= site_prograd.findAll('div',attrs = {'class':'subpages'})[0].findAll('a')

In [ ]:
# Agora será definida a estrutura do objeto json
def estrutura_json(sub,resp,tel,email):
  contatos_json =   [{"setor":"Prograd"},
                    {"resp":"{0}".format(resp)},
                    {"subsetor":"{0}".format(sub)},
                    {"fone":"{0}".format(tel)},
                    {"email":"{0}".format(email)},
                    {"site":"https://prograd.ufc.br/pt/"},
                    {"faq": "https://prograd.ufc.br/pt/perguntas-frequentes/"},
                    {"contats":"https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/"},
                    {"whatsapp":"+55(85)33669410"}]
  return contatos_json

In [ ]:
''''def transforma_dados(i,f,url):
  
  site = requisicao_site(url)
  contatos = site.findAll('p')
  tamanho = len(contatos[:f])
  dados = []
  for ind in range(i,tamanho):
    dados.append(contatos[ind].text.split('\n'))
  return dados

def estruturando_contatos(): # função específica
  vetor_armazena_contatos = [{"contatos":[]}]
  for ind in range(len(setores)):
    if ind == 0:
        contatos = transforma_dados(1,-2,str(setores[ind]["href"]))
        contato_principal = estrutura_json('Coordenação Prograd',contatos[ind][0],contatos[ind][2],contatos[ind][3])
        vetor_armazena_contatos[0]["contatos"].append(contato_principal)
        for posicao in range(1,len(contatos)):
          contatos_json = estrutura_json(contatos[posicao][0],contatos[posicao][1],contatos[posicao][2],contatos[posicao][3])
          vetor_armazena_contatos[0]["contatos"].append(contatos_json) 
    elif ind == 5:
        contatos = transforma_dados(0,-2,str(setores[5]["href"]))
        contato_principal = estrutura_json(contatos[0][0],contatos[2][0],contatos[2][2],contatos[2][3])
        vetor_armazena_contatos[0]["contatos"].append(contato_principal)
        for posicao in range(3,len(contatos)):
          if posicao !=3 and posicao!=6:                                                                                                            
            contatos_json = estrutura_json(contatos[posicao][0],contatos[posicao][1],contatos[posicao][2],contatos[posicao][3])
            vetor_armazena_contatos[0]["contatos"].append(contatos_json) 
          elif posicao==3:
            contatos_json = estrutura_json(contatos[3][0],contatos[3][1],contatos[3][2],'sem email')
            vetor_armazena_contatos[0]["contatos"].append(contatos_json) 
          elif posicao ==6:
            contatos_json = estrutura_json(contatos[posicao][0],contatos[posicao][2],contatos[posicao][1],contatos[posicao][3])
            vetor_armazena_contatos[0]["contatos"].append(contatos_json)   
    else:
        contatos = transforma_dados(0,3,str(setores[ind]["href"]))
        contatos_json = estrutura_json(contatos[0][0],contatos[2][0],contatos[2][2],contatos[2][3])
        vetor_armazena_contatos[0]["contatos"].append(contatos_json) 
    time.sleep(15)
  return vetor_armazena_contatos

prograd_json = estruturando_contatos()'''

'\'def transforma_dados(i,f,url):\n  \n  site = requisicao_site(url)\n  contatos = site.findAll(\'p\')\n  tamanho = len(contatos[:f])\n  dados = []\n  for ind in range(i,tamanho):\n    dados.append(contatos[ind].text.split(\'\n\'))\n  return dados\n\ndef estruturando_contatos(): # função específica\n  vetor_armazena_contatos = [{"contatos":[]}]\n  for ind in range(len(setores)):\n    if ind == 0:\n        contatos = transforma_dados(1,-2,str(setores[ind]["href"]))\n        contato_principal = estrutura_json(\'Coordenação Prograd\',contatos[ind][0],contatos[ind][2],contatos[ind][3])\n        vetor_armazena_contatos[0]["contatos"].append(contato_principal)\n        for posicao in range(1,len(contatos)):\n          contatos_json = estrutura_json(contatos[posicao][0],contatos[posicao][1],contatos[posicao][2],contatos[posicao][3])\n          vetor_armazena_contatos[0]["contatos"].append(contatos_json) \n    elif ind == 5:\n        contatos = transforma_dados(0,-2,str(setores[5]["href"]))\n 

In [ ]:
''' Gravando arquivo json
with open('contato_prograd.json','w') as file:
  arquivo = json.dump(contatos_prograd, file,ensure_ascii=False)'''

" Gravando arquivo json\nwith open('contato_prograd.json','w') as file:\n  arquivo = json.dump(contatos_prograd, file,ensure_ascii=False)"

In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

## Prograd(fixo) /Prae /Prointer
## Devido a regras muito distintas na estruturação do site de endereço e contatos da Prae e Prointer, além disso, o escopo pequeno de contatos esta parte da construção do json será feita manualmente.

In [ ]:
prae_json =[{'contatos_prae': [[{'setor': 'Prae'},
   {'resp': 'Pró-Reitora: Profa. Dra. Geovana Maria Cartaxo de Arruda Freire'},
   {'subsetor': 'Coordenação principal'},
   {'fone': '(85) 3366 7440'},
   {'email': 'prae.secretaria@ufc.br'},
   {'site': 'https://prae.ufc.br/pt/'},
   {'faq': 'Não existente'},
   {'contats': 'https://prae.ufc.br/pt/enderecos-e-telefones/'},
   {'whatsapp': '+55(85)3366 7440'}],
  [{'setor': 'Prae'},
   {'resp': 'Pró-Reitora: Profa. Dra. Geovana Maria Cartaxo de Arruda Freire'},
   {'subsetor': 'DIVISÃO DE APOIO ADMINISTRATIVO'},
   {'fone': '(85) 3366 7440'},
   {'email': 'prae.secretaria@ufc.br'},
   {'site': 'https://prae.ufc.br/pt/'},
   {'faq': 'Não existente'},
   {'contats': 'https://prae.ufc.br/pt/enderecos-e-telefones/'},
   {'whatsapp': '+55(85)3366 7440'}],
  [{'setor': 'Prae'},
   {'resp': 'Pró-Reitora: Profa. Dra. Geovana Maria Cartaxo de Arruda Freire'},
   {'subsetor': 'ASSESSORIA ADMINISTRATIVA'},
   {'fone': 'Fax: (85) 3366 7442'},
   {'email': 'praap.prae@ufc.br'},
   {'site': 'https://prae.ufc.br/pt/'},
   {'faq': 'Não existente'},
   {'contats': 'https://prae.ufc.br/pt/enderecos-e-telefones/'},
   {'whatsapp': '+55(85)3366 7440'}],
  [{'setor': 'Prae'},
   {'resp': 'Coordenadora: Márcia Regina Mariano de Sousa Arão'},
   {'subsetor': 'COORDENADORIA DE ASSISTÊNCIA ESTUDANTIL'},
   {'fone': '(85) 3366-7445 / 3366-7444'},
   {'email': 'case@ufc.br'},
   {'site': 'https://prae.ufc.br/pt/'},
   {'faq': 'Não existente'},
   {'contats': 'https://prae.ufc.br/pt/enderecos-e-telefones/'},
   {'whatsapp': '+55(85)3366 7440'}],
  [{'setor': 'Prae'},
   {'resp': 'Coordenadora: Márcia Regina Mariano de Sousa Arão'},
   {'subsetor': 'DIVISÃO DE ATENÇÃO AO ESTUDANTE'},
   {'fone': ' (85) 3366 7447'},
   {'email': 'dae.case@ufc.br'},
   {'site': 'https://prae.ufc.br/pt/'},
   {'faq': 'Não existente'},
   {'contats': 'https://prae.ufc.br/pt/enderecos-e-telefones/'},
   {'whatsapp': '+55(85)3366 7440'}],
  [{'setor': 'Prae'},
   {'resp': 'Coordenadora: Márcia Regina Mariano de Sousa Arão'},
   {'subsetor': 'DIVISÃO DE BENEFÍCIOS E MORADIA'},
   {'fone': '(85)3366-7448'},
   {'email': 'digeb.case@ufc.br'},
   {'site': 'https://prae.ufc.br/pt/'},
   {'faq': 'Não existente'},
   {'contats': 'https://prae.ufc.br/pt/enderecos-e-telefones/'},
   {'whatsapp': '+55(85)3366 7440'}],
  [{'setor': 'Prae'},
   {'resp': 'Coordenador: Wildner Lins de Souza'},
   {'subsetor': 'COORDENADORIA  DE ATIVIDADES DESPORTIVAS'},
   {'fone': '(85) 3366 7871'},
   {'email': 'desportoufc@ufc.br'},
   {'site': 'https://prae.ufc.br/pt/'},
   {'faq': 'Não existente'},
   {'contats': 'https://prae.ufc.br/pt/enderecos-e-telefones/'},
   {'whatsapp': '+55(85)3366 7440'}],
  [{'setor': 'Prae'},
   {'resp': 'Coordenador: Wildner Lins de Souza'},
   {'subsetor': 'DIVISÃO DE DESPORTO DE PARTICIPAÇÃO'},
   {'fone': '(85) 3366 7871'},
   {'email': 'desportoufc@ufc.br'},
   {'site': 'https://prae.ufc.br/pt/'},
   {'faq': 'Não existente'},
   {'contats': 'https://prae.ufc.br/pt/enderecos-e-telefones/'},
   {'whatsapp': '+55(85)3366 7440'}],
  [{'setor': 'Prae'},
   {'resp': 'Diretor: José Clovandi Costa Filho'},
   {'subsetor': 'DIVISÃO DE DESPORTO DE RENDIMENTO'},
   {'fone': '(85) 3366 7871'},
   {'email': 'desportoufc@ufc.br'},
   {'site': 'https://prae.ufc.br/pt/'},
   {'faq': 'Não existente'},
   {'contats': 'https://prae.ufc.br/pt/enderecos-e-telefones/'},
   {'whatsapp': '+55(85)3366 7440'}],
  [{'setor': 'Prae'},
   {'resp': 'Coordenador: Francisco José Albuquerque Cruz'},
   {'subsetor': 'COORDENADORIA DE RESTAURANTE UNIVERSITÁRIO'},
   {'fone': '(85) 3366 7441'},
   {'email': 'praeufc@gmail.com'},
   {'site': 'https://prae.ufc.br/pt/'},
   {'faq': 'Não existente'},
   {'contats': 'https://prae.ufc.br/pt/enderecos-e-telefones/'},
   {'whatsapp': '+55(85)3366 7440'}],
  [{'setor': 'Prae'},
   {'resp': 'Diretora: Natália Lopes Vasconcelos'},
   {'subsetor': 'DIVISÃO DE ALIMENTAÇÃO E NUTRIÇÃO'},
   {'fone': '(85) 3366 9531'},
   {'email': 'praeufc@gmail.com'},
   {'site': 'https://prae.ufc.br/pt/'},
   {'faq': 'Não existente'},
   {'contats': 'https://prae.ufc.br/pt/enderecos-e-telefones/'},
   {'whatsapp': '+55(85)3366 7440'}],
  [{'setor': 'Prae'},
   {'resp': 'Chefe: Conceição de Maria Coelho de Carvalho'},
   {'subsetor': 'Seção do Refeitório do Benfica'},
   {'fone': '(85) 3366 7776'},
   {'email': 'praeufc@gmail.com'},
   {'site': 'https://prae.ufc.br/pt/'},
   {'faq': 'Não existente'},
   {'contats': 'https://prae.ufc.br/pt/enderecos-e-telefones/'},
   {'whatsapp': '+55(85)3366 7440'}],
  [{'setor': 'Prae'},
   {'resp': 'Chefe: Cleonice de Castro Silva'},
   {'subsetor': 'Seção do Refeitório do Pici'},
   {'fone': '(85) 3366 9530'},
   {'email': 'praeufc@gmail.com'},
   {'site': 'https://prae.ufc.br/pt/'},
   {'faq': 'Não existente'},
   {'contats': 'https://prae.ufc.br/pt/enderecos-e-telefones/'},
   {'whatsapp': '+55(85)3366 7440'}],
  [{'setor': 'Prae'},
   {'resp': 'COORDENADORIA DE RESTAURANTE UNIVERSITÁRIO'},
   {'subsetor': 'Seção do Refeitório do Porangabuçu'},
   {'fone': ' (85) 3366 7447'},
   {'email': 'praeufc@gmail.com'},
   {'site': 'https://prae.ufc.br/pt/'},
   {'faq': 'Não existente'},
   {'contats': 'https://prae.ufc.br/pt/enderecos-e-telefones/'},
   {'whatsapp': '+55(85)3366 7440'}],
  [{'setor': 'Prae'},
   {'resp': 'Diretor: José Pereira Duarte'},
   {'subsetor': 'DIVISÃO DE SERVIÇOS OPERACIONAIS'},
   {'fone': ' (85) 3366 9530'},
   {'email': 'praeufc@gmail.com'},
   {'site': 'https://prae.ufc.br/pt/'},
   {'faq': 'Não existente'},
   {'contats': 'https://prae.ufc.br/pt/enderecos-e-telefones/'},
   {'whatsapp': '+55(85)3366 7440'}]]}]

In [ ]:
with open('contato_prae.json','w') as file:
  arquivo = json.dump(prae_json, file,ensure_ascii=False)

In [ ]:
prointer_json =[{'contatos_prointer': [[{'setor': 'Prointer'},
   {'resp': 'Augusto Albuquerque (Pró-Reitor)'},
   {'subsetor': 'Gabinete do Pró-Reitor'},
   {'fone': ' (85) 3366.7333'},
   {'email': 'augusto.albuquerque@ufc.br'},
   {'site': 'https://prointer.ufc.br/pt/seja-bem-vindo-a-prointer/'},
   {'faq': 'https://prointer.ufc.br/pt/relacoes-internacionais/perguntas-frequentes/'},
   {'contats': 'https://prointer.ufc.br/pt/sobre-a-prointer/endereco-e-telefones/'},
   {'whatsapp': ' (85) 3366.7333'}],
  [{'setor': 'Prointer'},
   {'resp': ['Camila Gueiros(Gestora Administrativa)',
     'Fábio Nogueira(Assessor)',
     'Bruno Souza(Atendimento ao Público)']},
   {'subsetor': 'Secretaria Administrativa'},
   {'fone': ' (85) 3366.7333'},
   {'email': ['camilagueiros@ufc.br',
     'fabionogueira@ufc.br',
     'secretaria@cai.ufc.br']},
   {'site': 'https://prointer.ufc.br/pt/seja-bem-vindo-a-prointer/'},
   {'faq': 'https://prointer.ufc.br/pt/relacoes-internacionais/perguntas-frequentes/'},
   {'contats': 'https://prointer.ufc.br/pt/sobre-a-prointer/endereco-e-telefones/'},
   {'whatsapp': ' (85) 3366.7333'}],
  [{'setor': 'Prointer'},
   {'resp': ['Ananda Badaró (Tradutora e Intérprete)',
     'Tadeu Azevedo(Tradutor e Intérprete)',
     'Document Verification Service']},
   {'subsetor': 'Equipe de Tradução'},
   {'fone': ' (85) 3366.7333'},
   {'email': ['traducao@cai.ufc.br',
     'tradutor@prointer.ufc.br',
     'documents@prointer.ufc.br']},
   {'site': 'https://prointer.ufc.br/pt/seja-bem-vindo-a-prointer/'},
   {'faq': 'https://prointer.ufc.br/pt/relacoes-internacionais/perguntas-frequentes/'},
   {'contats': 'https://prointer.ufc.br/pt/sobre-a-prointer/endereco-e-telefones/'},
   {'whatsapp': ' (85) 3366.7333'}],
  [{'setor': 'Prointer'},
   {'resp': ['Mônica Amorim(Diretora)',
     'Su Jianhua(Diretor)',
     'Luana Leite(Assistente de Administração']},
   {'subsetor': 'Instituto Confúcio da UFC'},
   {'fone': ' (85) 3366.9031'},
   {'email': ['monica_amorim@terra.com.br',
     'institutoconfucio@ufc.br',
     'institutoconfucio@ufc.br']},
   {'site': 'https://prointer.ufc.br/pt/seja-bem-vindo-a-prointer/'},
   {'faq': 'https://institutoconfucio.ufc.br/pt/'},
   {'contats': 'https://prointer.ufc.br/pt/sobre-a-prointer/endereco-e-telefones/'},
   {'whatsapp': ' (85) 3366.9031'}],
  [{'setor': 'Prointer'},
   {'resp': ['Rodrigo Rego(Coordenador)',
     'Fabiano Gadelha(Analista de Tecnologia da Informação)']},
   {'subsetor': 'Coordenadoria de Convênios Internacionais'},
   {'fone': ' (85) 3366.7336 '},
   {'email': ['convenios@prointer.ufc.br', 'fabiano.gadelha@sti.ufc.br']},
   {'site': 'https://prointer.ufc.br/pt/seja-bem-vindo-a-prointer/'},
   {'faq': 'https://prointer.ufc.br/pt/relacoes-internacionais/perguntas-frequentes/'},
   {'contats': 'https://prointer.ufc.br/pt/sobre-a-prointer/endereco-e-telefones/'},
   {'whatsapp': ' (85) 3366.7336 '}],
  [{'setor': 'Prointer'},
   {'resp': ['Talita Vasconcelos(Coordenadora) ',
     'Vanderleia Lucia de Souza(Assessora)']},
   {'subsetor': 'Coordenadoria de Mobilidade Acadêmica'},
   {'fone': '(85)3366.7336'},
   {'email': ['mobilidade@prointer.ufc.br', 'administrativo@cai.ufc.br']},
   {'site': 'https://prointer.ufc.br/pt/seja-bem-vindo-a-prointer/'},
   {'faq': 'https://prointer.ufc.br/pt/relacoes-internacionais/perguntas-frequentes/'},
   {'contats': 'https://prointer.ufc.br/pt/sobre-a-prointer/endereco-e-telefones/'},
   {'whatsapp': '(85)3366.7336'}],
  [{'setor': 'Prointer'},
   {'resp': ['Abraão Saraiva(Coordenador) ',
     'Gervina Brady(Assistente em Administração)',
     'Tatieures Gomes(Analista de Tecnologia da Informação)']},
   {'subsetor': 'Coordenadoria de Empreendedorismo'},
   {'fone': '(85) 3366.7335'},
   {'email': ['abraaofsjr@gmail.com',
     'gervina.brady@ufc.br',
     'tatieures@ufc.br']},
   {'site': 'https://prointer.ufc.br/pt/seja-bem-vindo-a-prointer/'},
   {'faq': 'https://prointer.ufc.br/pt/relacoes-internacionais/perguntas-frequentes/'},
   {'contats': 'https://prointer.ufc.br/pt/sobre-a-prointer/endereco-e-telefones/'},
   {'whatsapp': '((85) 3366.7335'}],
  [{'setor': 'Prointer'},
   {'resp': ['Bruno Matos(Coordenador)',
     'Patrícia Araújo(Administradora)',
     'Laura Figueiredo(Assistente em Administração)',
     'Ciro Texeira(Administrador)']},
   {'subsetor': 'Coordenadoria de Inovação Institucional'},
   {'fone': '  (85) 3366.7335'},
   {'email': ['brunomatos@ufc.br',
     'patricia.araujo@ufc.br|inovacao@prointer.ufc.br',
     'lauracfigueiredo@ufc.br',
     'ciro.teixeira@ufc.br']},
   {'site': 'https://prointer.ufc.br/pt/seja-bem-vindo-a-prointer/'},
   {'faq': 'https://prointer.ufc.br/pt/relacoes-internacionais/perguntas-frequentes/'},
   {'contats': 'https://prointer.ufc.br/pt/sobre-a-prointer/endereco-e-telefones/'},
   {'whatsapp': '((85) 3366.7335'}]]}]

In [ ]:
import json
with open('contato_prointer.json','w') as file:
  arquivo = json.dump(prointer_json, file,ensure_ascii=False)

### Para colocar na link da url /api/get_contact(setor:string) no flask basta criar o método GET para contatos e ,além disso, salvar cada uns dos jsons em uma lista, assim como será mostrado a seguir

In [ ]:
# Todos os contatos foram salvos em arquivo json, por isso irei somente colocar o carregar o arquivo do web scraping 
# e salvar no vetor juntos aos outros dois jsons.

In [ ]:
prograd_json = [{'contatos': 
                 [[{'setor': 'Prograd'},
    {'resp': 'Pró-Reitora: Profª Ana Paula de Medeiros Ribeiro'},
    {'subsetor': 'Coordenação Prograd'},
    {'fone': 'Fone: (85) 3366 9498'},
    {'email': 'E-mail: protocolo@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Responsável: Isabel Cristina Moraes de Souza Castro'},
    {'subsetor': 'Divisão de Apoio Administrativo'},
    {'fone': 'Fone: (85) 3366 9498'},
    {'email': 'E-mail: gabinete@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Responsável: Priscilla de Araújo Gois Pinheiro Guerra'},
    {'subsetor': 'Seção de Protocolo'},
    {'fone': 'Fone: (85) 3366 9410'},
    {'email': 'E-mail: protocolo@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Responsável: Mônica Cristina de Lucena Lucas'},
    {'subsetor': 'Divisão de Comunicação (Atendimento à imprensa)'},
    {'fone': 'Fone: (85) 3366 9527 (provisório)'},
    {'email': 'E-mail: comunicacao@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Assessor: Carlos César Osório de Melo'},
    {'subsetor': 'Assessoria de Legislação'},
    {'fone': 'Fone: (85) 3366 9421'},
    {'email': 'E-mail: ale@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Coordenador na UFC: Prof. Pedro Rogério'},
    {'subsetor': 'Programa de Residência Pedagógica'},
    {'fone': 'Fone: (85) 3366 9527'},
    {'email': 'E-mail: rp@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Coordenadora na UFC: Profª Maria José Costa dos Santos'},
    {'subsetor': 'Programa Institucional de Iniciação à Docência – PIBID'},
    {'fone': 'Fone: (85) 3366 9527'},
    {'email': 'E-mail: pibid@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Coordenador: Prof. Francisco Ari de Andrade'},
    {'subsetor': 'COORDENADORIA DE ACOMPANHAMENTO DISCENTE – CAD'},
    {'fone': 'Fone: (85) 3366 9519'},
    {'email': 'E-mail: cad@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Coordenador: Profa. Simone da Silveira Sá Borges'},
    {'subsetor': 'COORDENADORIA GERAL DE PROGRAMAS ACADÊMICOS – CGPA'},
    {'fone': 'Fone/WhatsApp: (85) 3366 9496'},
    {'email': 'E-mail: cgpa@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Coordenadora: Aline Batista de Andrade'},
    {'subsetor': 'COORDENADORIA DE PROJETOS E ACOMPANHAMENTO CURRICULAR – COPAC'},
    {'fone': 'Fone: (85) 3366 9526'},
    {'email': 'E-mail: copac@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Coordenadora: Profª Andréa Soares Rocha da Silva'},
    {'subsetor': 'COORDENADORIA DE PLANEJAMENTO E AVALIAÇÃO DE PROGRAMAS E AÇÕES ACADÊMICAS – COPAV'},
    {'fone': 'Fone: (85) 3366 9020'},
    {'email': 'E-mail: copav@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Coordenador: Prof. Rafael Bráz Azevedo Farias'},
    {'subsetor': 'COORDENADORIA DE PLANEJAMENTO, INFORMAÇÃO E COMUNICAÇÃO – COPIC'},
    {'fone': 'Fone: (85) 3366 9036'},
    {'email': 'E-mail: copic@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Diretora: Keyla Maciel Maia'},
    {'subsetor': 'DIVISÃO DE PLANEJAMENTO E ENSINO'},
    {'fone': 'Fone: (85) 3366 9036'},
    {'email': 'sem email'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Diretora: Milena Teixeira Barbosa'},
    {'subsetor': 'DIVISÃO DE SELEÇÃO'},
    {'fone': 'E-mail: disel@prograd.ufc.br'},
    {'email': 'Fones: (85) 3366 9522 / 3366 9423'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Diretor: Daélio Feijó Rabelo'},
    {'subsetor': 'DIVISÃO DE SELEÇÃO E MATRÍCULA'},
    {'fone': 'Fone: (85) 3366 9528'},
    {'email': 'E-mail: dsm@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Chefe: Francisco Ivanildo Ferreira Fialho'},
    {'subsetor': 'DIVISÃO DE MEMÓRIA E DOCUMENTAÇÃO'},
    {'fone': '(atendimento exclusivamente por e-mail)'},
    {'email': 'E-mail: dmd@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Responsável: Michelly Linhares de Moraes'},
    {'subsetor': 'Seção de Revalidação de Diplomas'},
    {'fone': 'Fone: (85) 3366 9521'},
    {'email': 'E-mail: revalidacao@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Chefe: Maria Cristina de Figueiredo Monteiro'},
    {'subsetor': 'Seção de Registro de Diplomas de Entidades Privadas'},
    {'fone': 'Fone: (85) 3366 9523'},
    {'email': 'E-mail: registroies@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Responsável: Maria de Fátima Andrade'},
    {'subsetor': 'Seção de Arquivo'},
    {'fone': 'Fone/WhatsApp: (85) 3366 9520'},
    {'email': 'E-mail: diarq@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': '+55(85)33669410'}]]}]

## Pondo link do whatsapp, por meio de regex

In [ ]:
def junta_string(vetor_string):
    texto = ''
    for elemento in vetor_string:
      texto += elemento
    return texto

In [ ]:
for elemento in prograd_json[0]['contatos']:        
    num = re.findall('[0-9]',elemento[8]['whatsapp'])
    elemento[8]['whatsapp'] = 'https://wa.me/'+junta_string(num)
prograd_json

[{'contatos': [[{'setor': 'Prograd'},
    {'resp': 'Pró-Reitora: Profª Ana Paula de Medeiros Ribeiro'},
    {'subsetor': 'Coordenação Prograd'},
    {'fone': 'Fone: (85) 3366 9498'},
    {'email': 'E-mail: protocolo@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': 'https://wa.me/558533669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Responsável: Isabel Cristina Moraes de Souza Castro'},
    {'subsetor': 'Divisão de Apoio Administrativo'},
    {'fone': 'Fone: (85) 3366 9498'},
    {'email': 'E-mail: gabinete@prograd.ufc.br'},
    {'site': 'https://prograd.ufc.br/pt/'},
    {'faq': 'https://prograd.ufc.br/pt/perguntas-frequentes/'},
    {'contats': 'https://prograd.ufc.br/pt/contatos/pro-reitor-e-apoio/'},
    {'whatsapp': 'https://wa.me/558533669410'}],
   [{'setor': 'Prograd'},
    {'resp': 'Responsável: Priscilla de 

In [ ]:
for elemento in prointer_json[0]['contatos_prointer']:        
    num = re.findall('[0-9]',elemento[8]['whatsapp'])
    elemento[8]['whatsapp'] = 'https://wa.me/'+junta_string(num)
prointer_json

[{'contatos_prointer': [[{'setor': 'Prointer'},
    {'resp': 'Augusto Albuquerque (Pró-Reitor)'},
    {'subsetor': 'Gabinete do Pró-Reitor'},
    {'fone': ' (85) 3366.7333'},
    {'email': 'augusto.albuquerque@ufc.br'},
    {'site': 'https://prointer.ufc.br/pt/seja-bem-vindo-a-prointer/'},
    {'faq': 'https://prointer.ufc.br/pt/relacoes-internacionais/perguntas-frequentes/'},
    {'contats': 'https://prointer.ufc.br/pt/sobre-a-prointer/endereco-e-telefones/'},
    {'whatsapp': 'https://wa.me/8533667333'}],
   [{'setor': 'Prointer'},
    {'resp': ['Camila Gueiros(Gestora Administrativa)',
      'Fábio Nogueira(Assessor)',
      'Bruno Souza(Atendimento ao Público)']},
    {'subsetor': 'Secretaria Administrativa'},
    {'fone': ' (85) 3366.7333'},
    {'email': ['camilagueiros@ufc.br',
      'fabionogueira@ufc.br',
      'secretaria@cai.ufc.br']},
    {'site': 'https://prointer.ufc.br/pt/seja-bem-vindo-a-prointer/'},
    {'faq': 'https://prointer.ufc.br/pt/relacoes-internacionais/pergun

In [ ]:
for elemento in prae_json[0]['contatos_prae']:        
    num = re.findall('[0-9]',elemento[8]['whatsapp'])
    elemento[8]['whatsapp'] = 'https://wa.me/'+junta_string(num)
prae_json

[{'contatos_prae': [[{'setor': 'Prae'},
    {'resp': 'Pró-Reitora: Profa. Dra. Geovana Maria Cartaxo de Arruda Freire'},
    {'subsetor': 'Coordenação principal'},
    {'fone': '(85) 3366 7440'},
    {'email': 'prae.secretaria@ufc.br'},
    {'site': 'https://prae.ufc.br/pt/'},
    {'faq': 'Não existente'},
    {'contats': 'https://prae.ufc.br/pt/enderecos-e-telefones/'},
    {'whatsapp': 'https://wa.me/558533667440'}],
   [{'setor': 'Prae'},
    {'resp': 'Pró-Reitora: Profa. Dra. Geovana Maria Cartaxo de Arruda Freire'},
    {'subsetor': 'DIVISÃO DE APOIO ADMINISTRATIVO'},
    {'fone': '(85) 3366 7440'},
    {'email': 'prae.secretaria@ufc.br'},
    {'site': 'https://prae.ufc.br/pt/'},
    {'faq': 'Não existente'},
    {'contats': 'https://prae.ufc.br/pt/enderecos-e-telefones/'},
    {'whatsapp': 'https://wa.me/558533667440'}],
   [{'setor': 'Prae'},
    {'resp': 'Pró-Reitora: Profa. Dra. Geovana Maria Cartaxo de Arruda Freire'},
    {'subsetor': 'ASSESSORIA ADMINISTRATIVA'},
    {'fone

# Localização do setores(API)

 api/get_location(nome_local:string, setor:string)
- Recebe nome(title) do local e setor e busca no banco de dados
- Saída: Json
{            
            
                "location": [
                    {"setor": "Prograd"},
                    {"latitude": "31.15254"},
                    {"longitude": "-3.51515"},
                    {"address": "Rua das carapiras, 2168"},
                    {"title": "Sala 90"},
                    {"desc": "Prédio com escadas..."},
                    {"maps": "https://www.google.com/maps/sdf61s"}
                 ]
            }}

In [ ]:
def struture_location(setor,lat,log,address,title,desc,link_maps):
              location = {"location": 
              [{"setor": setor },
              {"latitude": "{0}".format(lat)},
              {"longitude": "{0}".format(log)},
              {"address": address},
              {"title": title},
              {"desc": desc},
              {"maps": link_maps}]}
              return location

In [ ]:
# Prointer
desc_prointer = "..." # descrição de onde fica a prointer
loc_prointer = struture_location("Prointer",
                  -3.741827464215639, 
                  -38.53896668465892,
                  "Av. da Universidade, 2853 - Benfica - 60020-181 - Fortaleza – CE – Brasil",
                  "Prédio da Reitoria, Altos",
                  desc_prointer,
                  "https://goo.pgl/maps/nTJSJDkXaz8C2iF46")

# Instituto Confúcio 

desc_confucio = "..." 
loc_confucio = struture_location("Prointer",
                  -3.7394130073747687, -38.56946437265177,
                  "Rua Dr. Abdenago Rocha Lima, s/n - Campus do Pici, Fortaleza - CE, 60440-554",
                  "Instituto Confúcio",
                  desc_confucio,
                  "https://goo.gl/maps/vrmHuB9eKkbrtXxv8")


In [ ]:
# Prae
desc_prae = "..."
loc_prae = struture_location("Prae",
                  -3.741597800417145,
                  -38.53828905915975,
                  "Av. da Universidade, 2853 - Benfica",
                  "Edifício",
                  desc_prae,
                  "https://goo.gl/maps/2aiPH34CiDZRGfgt7")
# COORDENADORIA  DE ATIVIDADES DESPORTIVAS
desc_esportiva = "Quadra do CEU ao lado do Bloco de Psicologia"
loc_esportiva = struture_location("Prae",
                  -3.7399934596046998,
                  -38.53890611868392,
                  "Av. da Universidade, 2762 - Centro, Fortaleza - CE, 60020-181",
                  "Coordenadoria de atividade desportivas ",
                  desc_esportiva,
                  "https://goo.gl/maps/3tM7RX9oTqUTgUiz6")

In [ ]:
loc_prae

{'location': [{'setor': 'Prae'},
  {'latitude': '-3.741597800417145'},
  {'longitude': '-38.53828905915975'},
  {'address': 'Av. da Universidade, 2853 - Benfica'},
  {'title': 'Edifício'},
  {'desc': '...'},
  {'maps': 'https://goo.gl/maps/2aiPH34CiDZRGfgt7'}]}

In [ ]:
desc_prograd = "A frente da Biblioteca Central existe uma escada que leva abaixo da mesma,siga por ela para chegar atrás da bibioteca,ali você encontrará encontrará a Prograd."
loc_prograd = struture_location("Prograd",
                  -3.7425018781683863,
                  -38.574071941963766,
                  "Campus do Pici - Pici, Fortaleza - CE, 60455-760",
                  "Rua Biblioteca Central",
                  desc_prograd,
                  "https://goo.gl/maps/cWz3Kx4Yo5GKMGZq5")

In [ ]:
loc_prograd

{'location': [{'setor': 'Prograd'},
  {'latitude': '-3.7425018781683863'},
  {'longitude': '-38.574071941963766'},
  {'address': 'Campus do Pici - Pici, Fortaleza - CE, 60455-760'},
  {'title': 'Rua Biblioteca Central'},
  {'desc': 'A frente da Biblioteca Central existe uma escada que leva abaixo da mesma,siga por ela para chegar atrás da bibioteca,ali você encontrará encontrará a Prograd.'},
  {'maps': 'https://goo.gl/maps/cWz3Kx4Yo5GKMGZq5'}]}

# API

In [ ]:
from flask import Flask, jsonify
from flask_restful import Api, Resource, reqparse
from flask_ngrok import run_with_ngrok
from flask import make_response



import random
app = Flask(__name__)
api = Api(app)


# Scraping...

menu_ru_fort = checagem_de_retorno()
menu_ru_quixada = cardapio_quixada
contatos_prograd = prograd_json
contatos_prointer = prointer_json
contatos_prae = prae_json
localizacao_prograd = [loc_prograd]
localizacao_prointer = [loc_prointer,loc_confucio]
localizacao_prae = [loc_prae,loc_esportiva]

# Rota GET ...get_ru/<string:campi>"
class get_ru(Resource):
    def get(self,campi):
      self.campi = campi
      if campi == 'pici':
          return menu_ru_fort
      elif campi == 'quixada':
          return menu_ru_quixada
# Rota GET...get_contatos/<string:contatos>"

class get_contatos(Resource):
    def get(self,setor):
      self.setor = setor
      if self.setor == 'prograd':
        return contatos_prograd
      if self.setor ==  'prointer':
        return contatos_prointer
      if self.setor == 'prae':
        return contatos_prae

# Rota GET...get_location/<setor:location>
class get_location(Resource):
    def get(self,loc):
      self.loc = loc
      if self.loc == 'prograd':
        return localizacao_prograd
      elif self.loc == 'prointer':
        return localizacao_prointer
      elif self.loc == 'prae':
        return localizacao_prae
    
api.add_resource(get_ru, "/get_ru/<string:campi>") # passagem da url : /get_ru/(nome do campi escrito na função)
api.add_resource(get_contatos,"/get_contatos/<string:setor>")
api.add_resource(get_location,"/get_location/<string:loc>")


if __name__ == '__main__':
    run_with_ngrok(app)
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c8b5-35-204-201-56.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [21/Oct/2022 17:48:33] "GET /get_location/prae HTTP/1.1" 200 -
